# 용량 축소 및 피클로 저장

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 디렉토리 예시
# train데이터는 2020_Jane_Street_Market_Prediction 폴더 아래 input 폴더에 있다고 가정함

path = "drive/My Drive/Colab Notebooks/kaggle/2020_Jane_Street_Market_Prediction/"

In [3]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 999)

import warnings
warnings.filterwarnings("ignore")

In [4]:
def reduce_mem_usage(df):
    
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

def import_data(file):
  """create a dataframe and optimize its memory usage"""
  df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
  df = reduce_mem_usage(df)
  return df

In [6]:
print('-' * 80)
print('train')
train = import_data(path + 'input/train.csv')

--------------------------------------------------------------------------------
train
Memory usage of dataframe is 2516.843978881836 MB
Memory usage of dataframe after reduction 631.4908361434937 MB
Reduced by 74.90941665664761 % 


In [7]:
import pickle

with open(path+'input/train.pickle', 'wb') as handle:
    pickle.dump(train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(train.shape)
train.head()

In [9]:
del train

In [11]:
with open(path+'input/train.pickle', 'rb') as handle:
    train = pickle.load(handle)

In [ ]:
print(train.shape)
train.head()